## Purpose

This notebook shows how to get started with lightGBM in python.

I put together some work here https://www.kaggle.com/ca3295/glm-building-in-r-part-1 about how you might build a GLM in R. In some situations GLMs will be preferred over more-sophisticated methods despite reduced predictive performance. Having explored this data, I don't think this will be the case here. I should think the lightGBM approach will be far better.

Lets see!

In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split

Read the data. We did some EDA in the R notebook and I won't repeat here.

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
train.head()

Prepare the data for modelling. I'm going to keep this simple and have one validation fold. It's better to have multiple folds but this can take a long time. People also like to average predictions over multiple folds - which works very nicely but isn't the **key** part of what is happening here.

The key point is that if tree-based models are allowed too many iterations on the data (i.e. too many trees are built), they will start fitting noise. We don't want that, so after each new tree is built using the training data we check if including that tree improves predictive accuracy on data never seen before - i.e. the validation data. If it stops improving performance, then we stop building trees..

I'll keep 20% of the data back for validation.

In [ ]:
features=[f for f in train.columns if f not in ['id', 'target']]
features

In [ ]:
#train_test_split is a sklearn function for splitting your data. See https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_val, y_train, y_val = train_test_split(
    train[features], train['target'], test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
#Convert the data to a format lightGBM likes

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_val, y_val)



#Set up some parameters - we'll keep it simple for now
params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "max_depth": 3,
    'predict_contrib' : True,
    "learning_rate":0.05,
    "seed":22
}
#Run the model!
lgb_simple = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    verbose_eval=100,
    num_boost_round=99999,
    early_stopping_rounds=100
)

This looks promising. We were well above 0.7 in the GLM example (albeit we ontly used two features - it would have been exhausting to use more!)

Before submitting, let's have a look at what the model is doing.... we'll use SHAP to look under the hood a bit

The first plot ranks the feature in order of importance.

For each feature (row):

* The colour of each point reflects the value of the feature in the row of data being looked at (cold  = low value, hot = high value)
* The position of the point (left -> right) shows the impact on the prediction

So for cont1 we can see that higher values tend to mean lower predictions (hot colours are left)

In [ ]:
import shap
shap_values = shap.TreeExplainer(lgb_simple).shap_values(X_train[0:10000])
shap.summary_plot(shap_values, X_train[0:10000], feature_names=X_train.columns, max_display=200)

We can drill down some more. Here we look at how each feature contributes in more detail.

This time the value of the feature varies along the x-axis and the contribution to the prediction on the y-axis. The colour shows the impact of what SHAP deems to be a key interaction. For the second plot (cont7) the colder points trace a different shape to the hotter points - the "hotness" is the value of cont13, and this suggests there is an interaction between the two.

For more on SHAP plots see the notebook here https://github.com/slundberg/shap

In [ ]:
shap.dependence_plot("cont1", shap_values, X_train[0:10000])


In [ ]:
shap.dependence_plot("cont7", shap_values, X_train[0:10000])


Let's make predictions and submit.

In [ ]:
test['target']=lgb_simple.predict(test[features])

submission=test[['id', 'target']]
submission.head()

In [ ]:
submission.to_csv('lightGBM_simple_preds.csv', index=False)

This scored 0.70243, which at the time of writing is about in the middle of the leaderoard. As expected, this is a tangible improvement over the half-hearted GLM-attempt (0.72910) for much less effort.

There's way more to do here - there will be some simple gains to be made from hyperparameter tuning (the set we're using here is my usual starting parameters - no attempt at optimisation), and some from model ensembling. Feature engineering wins these kinds of competitions though.... my sense is the data could be a bit "bland" to engineer any nice features the model can't find itself - but have a go and see what you can find!

Note though that even the lightGBM model here (or the model at the top of the leaderboard at the time of writing with a score of 0.69534) isn't **that** much better than using the mean (0.73495 - note that these are simple RMSE metrics and so are fairly easy to interpret). In reality you would want to consider whether the effort of maintaining a model and delivering predictions is worth the increase in performance over the mean method. You might be in a domain where each 0.001 improvement is worth £ms, or you might find it just isn't worth it and you should pick a different battle - either way it's important to know!